In [ ]:

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

BASE = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
GEO_DIR = BASE / "geo"
OUTPUTS = BASE / "outputs"
MAPS = OUTPUTS / "maps"
PLOTS = OUTPUTS / "plots"
MAPS.mkdir(parents=True, exist_ok=True)
PLOTS.mkdir(parents=True, exist_ok=True)


In [ ]:

grid = gpd.read_file(GEO_DIR / "dwarka_grid_risk.geojson")
fig, ax = plt.subplots(1,1, figsize=(8,6))
grid.plot(column='mean_hi', cmap='hot', legend=True, ax=ax)
ax.set_title("Dwarka - Mean Heat Index by Grid Cell")
plt.tight_layout()
plt.savefig(PLOTS / "dwarka_mean_hi_choropleth.png")
logging.info("Saved plot %s", PLOTS / "dwarka_mean_hi_choropleth.png")


In [ ]:

import folium
from folium.plugins import HeatMap

m = folium.Map(location=[28.65,77.075], zoom_start=12, tiles="cartodbpositron")

folium.Choropleth(
    geo_data=str(GEO_DIR / "dwarka_grid_risk.geojson"),
    data=grid,
    columns=['cell_id','mean_hi'],
    key_on='feature.properties.cell_id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean Heat Index'
).add_to(m)
m.save(MAPS / "dwarka_choropleth.html")
logging.info("Saved interactive choropleth %s", MAPS / "dwarka_choropleth.html")


In [ ]:

streamlit_code = """
# === Streamlit app skeleton - save as src/streamlit_app.py ===
import streamlit as st
import geopandas as gpd
import folium
from streamlit_folium import st_folium

st.title("Dwarka Heat Risk Dashboard")
grid = gpd.read_file("geo/dwarka_grid_risk.geojson")
m = folium.Map(location=[28.65,77.075], zoom_start=12, tiles="cartodbpositron")
folium.Choropleth(
    geo_data="geo/dwarka_grid_risk.geojson",
    data=grid,
    columns=['cell_id','mean_hi'],
    key_on='feature.properties.cell_id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean Heat Index'
).add_to(m)
st_folium(m, width=700)
"""
(Path(BASE / "src" / "streamlit_app.py").parent).mkdir(parents=True, exist_ok=True)
with open(BASE / "src" / "streamlit_app.py", "w") as f:
    f.write(streamlit_code)
logging.info("Wrote Streamlit skeleton to src/streamlit_app.py")
